Importaciones 

In [13]:
from tensorflow import keras
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import set_random_seed
from tensorflow.keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Establecer semilla para reproducibilidad
set_random_seed(42)

Definir Directorios de Datos

In [14]:
# Rutas de los datasets
train_dir = "/workspaces/TensorFlow-Pet-Classifier/data/train"
test_dir = "/workspaces/TensorFlow-Pet-Classifier/data/test"


Preparar Generadores de Imágenes

In [15]:

# Generador de imágenes de entrenamiento con Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Generador de imágenes de test (solo normalización)
test_datagen = ImageDataGenerator(rescale=1./255)

Cargar Imágenes de Entrenamiento

In [16]:
# Cargar imágenes de entrenamiento (separadas por carpetas)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary'
)

Found 22030 images belonging to 2 classes.


Cargar Imágenes de Test

In [17]:
# Preparar DataFrame para test, ya que las imágenes no tienen carpetas
test_images = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]
test_df = pd.DataFrame({'filename': test_images})

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col="filename",
    y_col=None,  # No hay etiquetas
    target_size=(200, 200),
    batch_size=32,
    class_mode=None,  # Sin etiquetas
    shuffle=False  # Para que coincida con el orden de predicciones
)

Found 12500 validated image filenames.


Definir el Modelo 

In [18]:
# Definir el modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPool2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Definir Callbacks

In [19]:
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='auto')
early = EarlyStopping(monitor='val_accuracy', patience=5, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

Entrenar el Modelo

In [20]:
# Entrenamiento del modelo
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=test_generator,
    validation_steps=len(test_generator),
    callbacks=[checkpoint, early, reduce_lr]
)

/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
528/689 ━━━━━━━━━━━━━━━━━━━━ 2:21 878ms/step - accuracy: 0.5890 - loss: 0.6714

KeyboardInterrupt: 

Evaluar el Modelo

In [ ]:
# Evaluación del modelo
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('\nTest accuracy:', test_acc)

Visualizar el Rendimiento del Modelo

In [ ]:
# Graficar el rendimiento del modelo
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Model Performance Over Epochs")
plt.ylabel("Value")
plt.xlabel("Epoch")
plt.legend()
plt.show()

Hacer Predicciones en el Conjunto de Test

In [ ]:
# Predicción sobre el conjunto de test
predictions = model.predict(test_generator)
predictions = [1 if p > 0.5 else 0 for p in predictions]  # Convertir a 0 (gato) o 1 (perro)

Visualizar Predicciones

In [12]:
# Visualización de imágenes con predicciones
def visualize_predictions(model, test_generator, num_images=25):
    filenames = test_generator.filenames
    indices = np.random.choice(len(filenames), num_images, replace=False)
    
    plt.figure(figsize=(20, 20))
    for i, idx in enumerate(indices):
        img_path = os.path.join(test_dir, filenames[idx])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        pred_label = "Dog" if predictions[idx] == 1 else "Cat"
        
        plt.subplot(5, 5, i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Prediction: {pred_label}")

    plt.show()

# Llamada a la función para visualizar las predicciones
visualize_predictions(model, test_generator, num_images=25)


Found 22030 images belonging to 2 classes.
Found 12500 validated image filenames.


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-28 22:40:37.485070: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 38/689 ━━━━━━━━━━━━━━━━━━━━ 12:37 1s/step - accuracy: 0.5036 - loss: 0.7255

KeyboardInterrupt: 